<a href="https://colab.research.google.com/github/jeongjin-shin/Cartpole-SARSA/blob/main/cartpole_sarsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import gym
from tensorflow.keras.layers import *
import random
import math

In [2]:
model = tf.keras.models.Sequential()

model.add(Dense(12,activation = 'tanh',input_shape = (4,)))
model.add(Dense(2,activation = 'linear')) 

In [3]:
model.compile(optimizer = 'adam',loss = 'MSE',metrics = ['accuracy'])

In [4]:
episodes = 1000
env = gym.make("CartPole-v1")

In [5]:
def preprocessing(state):
    return np.reshape(state,(1,4))

In [6]:
def get_learning_rate(t):
        return max(0.1, min(1., 1. - math.log10((t + 1) / 25)))

In [7]:
model= tf.keras.models.load_model("cartpole-sarsa1541.h5")

In [9]:
replay_mem = []
replay_size = 32
reward_list = []
local_mean = list()
newaction = 0
for episode in range(1541,2000):

    done = False
    state = env.reset()
    count = 0
    reward_tot = 0
    model.save("cartpole-sarsa.h5")

    while not done:
        count += 1
        e = (1/(episode/200+1))
        
        
        #epslion greedy search
        Q = model.predict(preprocessing(state))
        if e>np.random.rand(1):
            action = env.action_space.sample()
        else:
            action = np.argmax(Q)
            
            
        #take_action and set reward
        state_next, reward, done, info = env.step(action)
        if done:
            reward = - 100
            
        
        #replay_mem
        replay_mem.append([state,action,reward,state_next,done])
        
        if len(replay_mem)>2048:
            del replay_mem[0]
            
            
        state = state_next
        reward_tot += reward
        state = state_next
        
        Q_list = []
        state_list = []
        
        
        #set this_replay_Size
        if len(replay_mem)<replay_size:
            this_replay_size = len(replay_mem)
        else:
            this_replay_size = replay_size
            
            
        #sample random batch from replay_memory
        for sample in random.sample(replay_mem,this_replay_size):
            state_m,action_m,reward_m,state_next_m,done_m = sample
            if done:
                Q[0,action] = reward_m
            else:
                Q_new = model.predict(preprocessing(state_next_m))
                if e>np.random.rand(1):
                  new_action = env.action_space.sample()
                else:
                  new_action = np.argmax(Q)
            
                Q[0,action] += get_learning_rate(e)*(reward+0.9*Q_new[0,new_action])
            Q_list.append(Q)
            state_list.append(state_m)
              
        #convert to nupmy array and train
        Q_list = np.array(Q_list)        
        state_list = np.array(state_list)
        
        hist = model.fit(state_list,Q_list,epochs = 5,verbose = 0)
        #print("Done :",done,"     Reward :",reward,"     Reward_total :",reward_tot)
    
    local_mean.append(reward_tot)
    reward_list.append(reward_tot)
    if episode%10 == 0:
        print("Episode :",episode+1,"     Reward_total :", reward_tot,"     Reward_local_mean :",np.mean(local_mean))
        local_mean = list()
        
print("*******End Learning")

Episode : 1551      Reward_total : -92.0      Reward_local_mean : -91.1
Episode : 1561      Reward_total : -92.0      Reward_local_mean : -91.3
Episode : 1571      Reward_total : -92.0      Reward_local_mean : -90.7
Episode : 1581      Reward_total : -90.0      Reward_local_mean : -91.2
Episode : 1591      Reward_total : -90.0      Reward_local_mean : -90.9
Episode : 1601      Reward_total : -92.0      Reward_local_mean : -91.5
Episode : 1611      Reward_total : -91.0      Reward_local_mean : -90.1
Episode : 1621      Reward_total : -91.0      Reward_local_mean : -91.1
Episode : 1631      Reward_total : -91.0      Reward_local_mean : -90.7
Episode : 1641      Reward_total : -90.0      Reward_local_mean : -90.7
Episode : 1651      Reward_total : -90.0      Reward_local_mean : -91.7
Episode : 1661      Reward_total : -92.0      Reward_local_mean : -90.0
Episode : 1671      Reward_total : -91.0      Reward_local_mean : -90.8
Episode : 1681      Reward_total : -91.0      Reward_local_mean 